In [1]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
import json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [2]:
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [3]:
session = get_session('EMPLOYEE',"fd0cf79c-8118-43a7-8f0b-059e8f78227a")

In [5]:
from snowflake.ml.registry.registry import Registry

In [ ]:

  "configuration": {"filter_condition": "",
  "frequency": {"frequencyType":"ONCE"},
  "input_dataset": "EMPLOYEE_TEST",
  "write_strategy": "New Table"
},
  "deployment_type": "BATCH",
  "model_name": "MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_PRAKHAR_TEST",
  "version": "V_110405"
}

In [15]:
reg = Registry(session=session)
m = reg.get_model('MODEL_5AABC4FE_FC3D_4121_9E11_52C83C9FECC9_FDC_PRAKHAR_TEST')
mv = m.version('V_110405')
# fetch_data_sql = "select * from " + input_table + " " + filter_cond
df = session.sql('select * from EMPLOYEE_TEST').to_pandas()
df.apply(lambda s:s.str.replace('"', ""))
print(type(df))
remote_prediction = mv.run(df, function_name="predict")
new_table_name = "prediction_" + 'output_table_batch'
session.write_pandas(remote_prediction, new_table_name, auto_create_table=True, overwrite=True)

AttributeError: Can only use .str accessor with string values!